In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

In [27]:
from sklearn.datasets import load_boston
from sklearn.cross_validation import train_test_split

# 讀入Datasets

In [28]:
data = load_boston()

In [29]:
feature = data.data
target = data.target

In [30]:
data.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [31]:
df = pd.DataFrame(feature, columns=data.feature_names)
df.head(2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.9,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14


In [32]:
df.shape

(506, 13)

In [34]:
x_train, x_test, y_train, y_test = train_test_split(df, target, train_size=0.8, random_state = 42)

# 建立input function 

In [35]:
def train_input_fn(feature, label, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(feature), label))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

# 建立Feature columns

In [36]:
feature_columns = []
for key in df.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
feature_columns

[_NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(ke

In [38]:
os.path.abspath('DNNregressor')

'C:\\Users\\AdamChang\\Documents\\Python\\練習區\\DNNregressor'

# 初始化model


In [39]:
model = tf.estimator.DNNRegressor(feature_columns=feature_columns,
                                 hidden_units=[64,32,10],
                                 label_dimension=1,
                                 optimizer='Adagrad',
                                 activation_fn=tf.nn.relu,
                                 dropout=None,
                                 model_dir=os.path.abspath('DNNregressor'),
                                 loss_reduction=tf.losses.Reduction.SUM)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C81B2DCEF0>, '_master': '', '_train_distribute': None, '_service': None, '_save_summary_steps': 100, '_task_type': 'worker', '_save_checkpoints_steps': None, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'C:\\Users\\AdamChang\\Documents\\Python\\練習區\\DNNregressor', '_is_chief': True, '_num_ps_replicas': 0, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_global_id_in_cluster': 0}


# Model training 


In [40]:
model.train(input_fn=lambda:train_input_fn(x_train, y_train,32), steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\AdamChang\Documents\Python\練習區\DNNregressor\model.ckpt.
INFO:tensorflow:loss = 10639.783, step = 1
INFO:tensorflow:global_step/sec: 485.185
INFO:tensorflow:loss = 2009.9749, step = 101 (0.206 sec)
INFO:tensorflow:global_step/sec: 659.634
INFO:tensorflow:loss = 1353.2529, step = 201 (0.156 sec)
INFO:tensorflow:global_step/sec: 349.971
INFO:tensorflow:loss = 1808.0232, step = 301 (0.289 sec)
INFO:tensorflow:global_step/sec: 549.704
INFO:tensorflow:loss = 2394.7126, step = 401 (0.178 sec)
INFO:tensorflow:global_step/sec: 489.111
INFO:tensorflow:loss = 2771.902, step = 501 (0.203 sec)
INFO:tensorflow:global_step/sec: 657.266
INFO:tensorflow:loss = 1606.3169, step = 601 (0.153 sec)
INFO:tensorflow:global

# model testing 

In [47]:
result = model.evaluate(input_fn=lambda: train_input_fn(x_test, y_test, 32),steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-03-06:04:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\AdamChang\Documents\Python\練習區\DNNregressor\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-06-03-06:04:52
INFO:tensorflow:Saving dict for global step 1000: average_loss = 46.139877, global_step = 1000, loss = 1476.4761


In [48]:
result

{'average_loss': 46.139877, 'global_step': 1000, 'loss': 1476.4761}